In [1]:
import json

In [3]:
with open('train.json', 'r', encoding='utf8') as f:
    train = json.load(f)

In [14]:
utterance_concat = []
for dialog in train:
    dialog_utterance = []
    for window in dialog:
        dialog_utterance.append(window['utterances'][-1])
    utterance_concat.append(dialog_utterance)

In [21]:
def _merge(previous_statuses_w, current_statuses_w):
    if '阳性' in previous_statuses_w and '阴性' in current_statuses_w:
        previous_statuses_w.remove('阳性')
    if '阴性' in previous_statuses_w and '阳性' in current_statuses_w:
        previous_statuses_w.remove('阴性')
    if '医生阳性' in previous_statuses_w and '医生阴性' in current_statuses_w:
        previous_statuses_w.remove('医生阳性')
    if '医生阴性' in previous_statuses_w and '医生阳性' in current_statuses_w:
        previous_statuses_w.remove('医生阴性')
    if '未知' in previous_statuses_w and len(current_statuses_w) > 0:
        previous_statuses_w.remove('未知')
    if len(previous_statuses_w) > 0 and '未知' in current_statuses_w:
        current_statuses_w.remove('未知')
    merged_statuses_w = previous_statuses_w | current_statuses_w
    return merged_statuses_w
    
def _construct_prefixs(labels):
    prefixs = dict()
    for label in labels:
        prefix, status = label.split('-')
        status = status.split(':')[-1]
        try:
            prefixs[prefix].add(status)
        except KeyError:
            prefixs[prefix] = {status}
    return prefixs

def merge(previous_labels_w, current_labels_w):
    previous_prefixs = _construct_prefixs(previous_labels_w)
    current_prefixs = _construct_prefixs(current_labels_w)
    for key in current_prefixs.keys():
        if key not in previous_prefixs:
            previous_prefixs[key] = current_prefixs[key]
        else:
            previous_prefixs[key] = _merge(previous_prefixs[key], current_prefixs[key])
    merged_labels_w = set()
    for key in previous_prefixs.keys():
        for status in previous_prefixs[key]:
            merged_labels_w.add('{}-{}'.format(key, status))
    return merged_labels_w


In [29]:
label_concat = []
for dialog in train:
    dialog_label = set()
    for window in dialog:
        dialog_label = merge(dialog_label, set(window['label']))
    label_concat.append(list(dialog_label))

In [30]:
dialog_with_label = []
for i in range(len(label_concat)):
    dialog_with_label.append({'utterance': utterance_concat[i], 'label': label_concat[i]})

In [46]:
output = json.dumps(dialog_with_label,ensure_ascii=False)

In [52]:
with open('D:/NLPCode/MIE-ACL-2020/concatenated.json', 'w') as outfile:
    outfile.write(output)
